# Model Storage: Preservation and Reuse 

Storing data is often tricky. You may be wondering what output to save, what output to throw away. You may also have questions surround what formats to use to store your data. 

PICASO's recommended model output is `xarray`. There are several `xarray` tutorials throughout the picaso documentation. [The most thorough is in the data uniformity tutorial](https://natashabatalha.github.io/picaso/notebooks/codehelp/data_uniformity_tutorial.html). 

Here you will learn how to use a nifty picaso function that will store your the **minimum data needed to reproduce your 1D spectrum**. The one caveat is that we will not store opacity data. However, you can always get your reference opacity data using the [auto citation tools](https://natashabatalha.github.io/picaso/notebooks/References.html). 

For this tutorial we assume you already know the basics of running 1D models. Note that we have still not enabled this functionality for 3D and 4D runs. Please contact the developers if you are interested in this functionality. 

In [1]:
import picaso.justdoit as jdi
import picaso.justplotit as jpi
u = jdi.u #astropy units

## Storing your run with xarray

Here we show a simple example of how to use the `output_xarray` function for preservation

In [2]:
opa = jdi.opannection(wave_range=[0.3,1])
pl = jdi.inputs()#calculation='brown')
pl.gravity(radius=1, radius_unit=u.Unit('R_jup'), 
           mass=1, mass_unit=u.Unit('M_jup'))
pl.atmosphere(filename=jdi.jupiter_pt(), sep='\s+')
pl.phase_angle(0)
pl.clouds(filename=jdi.jupiter_cld(), sep='\s+')
pl.star(opa, 5000,0,4, radius=1, radius_unit=u.Unit("R_sun"), semi_major=1, semi_major_unit=1*u.AU)
#MUST USE full output=True for this functionality
df= pl.spectrum(opa, calculation='reflected', full_output=True)


In [3]:
preserve = jdi.output_xarray(df,pl,savefile='/data/picaso_dbs/model.nc')

In [4]:
preserve

<xarray.Dataset> Size: 579kB
Dimensions:         (pressure: 61, wavelength: 12039, pressure_layer: 60,
                     wavenumber_layer: 196)
Coordinates:
  * pressure        (pressure) float64 488B 1e-06 1.41e-06 2e-06 ... 708.0 1e+03
  * wavelength      (wavelength) float64 96kB 0.9999 0.9998 ... 0.3001 0.3
Dimensions without coordinates: pressure_layer, wavenumber_layer
Data variables: (12/21)
    temperature     (pressure) float64 488B 150.9 149.7 ... 1.525e+03 1.675e+03
    albedo          (wavelength) float64 96kB 0.2771 0.2203 ... 0.7188 0.7187
    fpfs_reflected  (wavelength) float64 96kB 6.329e-08 5.031e-08 ... 1.642e-07
    e-              (pressure) float64 488B 4.5e-38 4.5e-38 ... 2.15e-11
    H2              (pressure) float64 488B 0.837 0.837 0.837 ... 0.836 0.836
    H               (pressure) float64 488B 4.5e-38 4.5e-38 ... 3.37e-06
    ...              ...
    NH3             (pressure) float64 488B 0.000137 0.000137 ... 0.000155
    N2              (pressure) float64 488B 5.42e-17 1.58e-17 ... 2.73e-05
    PH3             (pressure) float64 488B 4.5e-38 4.5e-38 ... 5.03e-07
    opd             (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
    ssa             (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
    asy             (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
Attributes:
    planet_params:   {"mp": {"value": 1.8981245973360504e+30, "unit": "g"}, "...
    stellar_params:  {"database": "ck04models", "steff": 5000, "feh": 0, "log...
    orbit_params:    {"sma": {"value": 14959787070000.0, "unit": "cm"}}

## Reusing your run with xarray

We often revisit models time and time again. Maybe you want to change your wavelength range, or observing geometry (e.g. transit vs emission). 

Here we show a simple example of how to use the `input_xarray` function for reuse. In this simple example, we will take our previous input but instead of computing reflected light, add thermal emission and transmission

In [5]:
opa = jdi.opannection(wave_range=[0.3,14])
ds = jdi.xr.load_dataset('/data/picaso_dbs/model.nc')
reuse = jdi.input_xarray(ds, opa)
new_model = reuse.spectrum(opa, calculation='reflected+thermal', full_output=True)

In [6]:
new_output= jdi.output_xarray(new_model, reuse)

In [7]:
new_output

<xarray.Dataset> Size: 1MB
Dimensions:        (pressure: 61, wavelength: 28717, pressure_layer: 60,
                    wavenumber_layer: 196)
Coordinates:
  * pressure       (pressure) float64 488B 1e-06 1.41e-06 2e-06 ... 708.0 1e+03
  * wavelength     (wavelength) float64 230kB 5.3 5.299 5.299 ... 0.3001 0.3
Dimensions without coordinates: pressure_layer, wavenumber_layer
Data variables: (12/22)
    temperature    (pressure) float64 488B 150.9 149.7 ... 1.525e+03 1.675e+03
    flux_emission  (wavelength) float64 230kB 1.84e+05 1.84e+05 ... 1.27e-08
    fpfs_emission  (wavelength) float64 230kB 1.776e-09 1.776e-09 ... 3.66e-24
    albedo         (wavelength) float64 230kB 0.09839 0.09843 ... 0.7188 0.7187
    e-             (pressure) float64 488B 4.5e-38 4.5e-38 ... 4.46e-12 2.15e-11
    H2             (pressure) float64 488B 0.837 0.837 0.837 ... 0.836 0.836
    ...             ...
    NH3            (pressure) float64 488B 0.000137 0.000137 ... 0.000155
    N2             (pressure) float64 488B 5.42e-17 1.58e-17 ... 2.73e-05
    PH3            (pressure) float64 488B 4.5e-38 4.5e-38 ... 5.11e-07 5.03e-07
    opd            (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
    ssa            (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
    asy            (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
Attributes:
    planet_params:   {"effective_temp": 19.585293614677756, "mp": {"value": 1...
    stellar_params:  {"database": "ck04models", "steff": 5000, "feh": 0, "log...

## Adding meta data

We have a tutorial that shows what some recommended meta data might be. `output_xarray` has a field `add_output` that allows you to add extra arguments to the `xarray.Dataset.attrs`. However, it has to be in a very specific format. 

You can see the basic format by running this function: 



In [8]:
jdi.standard_metadata()

{'author': 'optional',
 'contact': 'optional',
 'code': 'optional',
 'doi': 'optional',
 'planet_params': {'rp': 'usually taken from picaso',
  'mp': 'usually taken from picaso',
  'mh': 'optional',
  'cto': 'optional',
  'heat_redis': 'optional',
  'p_reference': 'usually taken from picaso',
  'tint': 'optional'},
 'stellar_params': {'logg': 'usually taken from picaso',
  'feh': 'usually taken from picaso',
  'steff': 'usually taken from picaso',
  'rs': 'usually taken from picaso',
  'ms': 'optional',
  'database': 'usually taken from picaso'},
 'orbit_params': {'sma': 'usually taken from picaso'}}

In [9]:
opa = jdi.opannection(wave_range=[0.3,1])
pl = jdi.inputs()#calculation='brown')
pl.gravity(radius=1, radius_unit=u.Unit('R_jup'), 
           mass=1, mass_unit=u.Unit('M_jup'))
mh = 0 
cto = 1

pl.atmosphere(filename=jdi.jupiter_pt(), sep='\s+') 
pl.chemeq_visscher(cto, mh)
pl.phase_angle(0)
pl.clouds(filename=jdi.jupiter_cld(), sep='\s+')
pl.star(opa, 5000,0,4, radius=1, radius_unit=u.Unit("R_sun"), semi_major=1, semi_major_unit=1*u.AU)
#MUST USE full output=True for this functionality
df= pl.spectrum(opa, calculation='reflected', full_output=True)

preserve = jdi.output_xarray(df,pl,add_output={
    'author':"Awesome Scientist",
    'contact' : "awesomescientist@universe.com",
    'code' : "picaso", 
    'planet_params':{'mh':mh, 'co':cto}, 
    'cloud_params':{'fsed':3}
})


In [10]:
preserve

<xarray.Dataset> Size: 589kB
Dimensions:         (pressure: 61, wavelength: 12039, pressure_layer: 60,
                     wavenumber_layer: 196)
Coordinates:
  * pressure        (pressure) float64 488B 1e-06 1.41e-06 2e-06 ... 708.0 1e+03
  * wavelength      (wavelength) float64 96kB 0.9999 0.9998 ... 0.3001 0.3
Dimensions without coordinates: pressure_layer, wavenumber_layer
Data variables: (12/40)
    temperature     (pressure) float64 488B 150.9 149.7 ... 1.525e+03 1.675e+03
    albedo          (wavelength) float64 96kB 0.2769 0.22 ... 0.7188 0.7188
    fpfs_reflected  (wavelength) float64 96kB 6.324e-08 5.025e-08 ... 1.642e-07
    e-              (pressure) float64 488B 4.5e-38 4.5e-38 ... 2.278e-11
    H2              (pressure) float64 488B 0.8359 0.8359 ... 0.8359 0.8358
    H               (pressure) float64 488B 4.5e-38 4.5e-38 ... 3.369e-06
    ...              ...
    MgH             (pressure) float64 488B 4.5e-38 4.5e-38 ... 2.843e-11
    OCS             (pressure) float64 488B 4.5e-38 4.5e-38 ... 7.931e-12
    graphite        (pressure) float64 488B 4.5e-38 4.5e-38 ... 4.5e-38 4.5e-38
    opd             (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
    ssa             (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
    asy             (pressure_layer, wavenumber_layer) float64 94kB 0.0 ... 0.0
Attributes:
    author:          Awesome Scientist
    code:            picaso
    contact:         awesomescientist@universe.com
    planet_params:   {"mp": {"value": 1.8981245973360504e+30, "unit": "g"}, "...
    stellar_params:  {"database": "ck04models", "steff": 5000, "feh": 0, "log...
    orbit_params:    {"sma": {"value": 14959787070000.0, "unit": "cm"}}
    cloud_params:    {'fsed': 3}